# Busca de dados - Copa\n
\n
Use este notebook para coletar os dados da Copa (16 times) e gerar os arquivos em `datasets_copa`.

In [1]:
import cartolafc
import pandas as pd
import json
import time
import requests
from functools import lru_cache

pd.set_option('display.max_columns', 50)            # permite a visualizacao de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os numeros aparecerem com duas casas decimais

# Cria uma instancia da API
api = cartolafc.Api(attempts=5)

# Constantes do 1o turno
INICIO_COPA = 1
FIM_COPA = 4
COLUNAS_RODADAS = [f"Rodada {r}" for r in range(INICIO_COPA, FIM_COPA + 1)]



2026-01-29 23:49:53,190 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
# Lista fixa de IDs dos participantes (32 times)
ids_participantes = [
                      25811332, 14124559, 28741323, 51010813, 29228373, 49180400, 3447341, 212042, 13913874, 3424598, 117598, 18344271, 48498051, 3708025, 25565675, 18223508,
                      24468241, 13951133, 24856400, 1747619, 20696550, 528730, 335716, 5823700, 18661583, 186283, 13707047, 18642587, 3851966, 18346776, 479510, 19033717
                    ]



In [3]:
HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json,text/plain,*/*",
    "Referer": "https://cartola.globo.com/",
}

@lru_cache(maxsize=5000)
def nome_time_por_id_api(time_id: int, timeout=15) -> str:
    endpoints = [
        f"https://api.cartolafc.globo.com/time/id/{time_id}",
        f"https://api.cartolafc.globo.com/time/{time_id}",
    ]

    for url in endpoints:
        for tentativa in range(3):
            try:
                r = requests.get(url, headers=HEADERS, timeout=timeout)
                if r.status_code == 429:
                    time.sleep(0.8 + tentativa * 0.8)
                    continue
                if r.status_code != 200:
                    break

                data = r.json()
                if isinstance(data, dict):
                    if isinstance(data.get("time"), dict) and isinstance(data["time"].get("nome"), str):
                        return data["time"]["nome"]
                    if isinstance(data.get("nome"), str):
                        return data["nome"]
                break
            except Exception:
                time.sleep(0.5)
                continue

    return f"Time {time_id}"

In [4]:
def _id_int(val):
    try:
        if val is None:
            return None
        if isinstance(val, str) and not val.strip():
            return None
        return int(val)
    except Exception:
        return None

# Base com todos os participantes (aceita vazios)
if not isinstance(ids_participantes, list):
    raise ValueError("ids_participantes precisa ser uma lista de IDs")

ids_participantes = list(dict.fromkeys(ids_participantes))
valid_ids = [v for v in (_id_int(x) for x in ids_participantes) if v is not None]

if valid_ids:
    df_base = pd.DataFrame({"time_id": valid_ids}).drop_duplicates()
    df_base["Time"] = df_base["time_id"].apply(nome_time_por_id_api)
    df_base = df_base.set_index("time_id").sort_index()
else:
    df_base = pd.DataFrame(columns=["Time"])
    df_base.index.name = "time_id"

# Dicionario Nome -> ID (compatibilidade com codigo legado)
ids_times_dict = {row["Time"]: row["time_id"] for _, row in df_base.reset_index().iterrows()} if not df_base.empty else {}

# Links para o Excel
if not df_base.empty:
    df_urls = pd.DataFrame({
        "Nome do Time": df_base["Time"].values,
        "ID do Time": df_base.index.values,
    })

    df_urls["Link do Time"] = df_urls["ID do Time"].apply(
        lambda x: f"https://cartola.globo.com/#!/time/{x}"
    )

    df_urls = df_urls[["Nome do Time", "ID do Time", "Link do Time"]]

    caminho_excel = "links_times_cartola_copa_leon.xlsx"
    df_urls.to_excel(caminho_excel, index=False)
    print(f"? Arquivo salvo com sucesso: {caminho_excel}")

display(df_base)
# display(df_urls)



? Arquivo salvo com sucesso: links_times_cartola_copa_leon.xlsx


,Time
time_id,
117598,A Lenda Super Vasco F.c
186283,FBC Colorado
212042,Tatols Beants F.C
335716,teves_futsal20 f.c
479510,TEAM LOPES 99
528730,Gremiomaniasm
1747619,JV5 Tricolor Gaúcho
3424598,TIGRE LEON
3447341,PUXE FC


In [5]:
def campeonato_comecou(ids, rodada_ref=INICIO_COPA):
    lista_ids = list(ids.values()) if isinstance(ids, dict) else list(ids)
    for time_id in lista_ids:
        try:
            t = api.time(time_id=time_id, rodada=rodada_ref)
            v = getattr(t, "ultima_pontuacao", None)
            if v is not None:
                return True
        except Exception:
            continue
    return False

In [6]:
# ================================
# Pontuacoes (parciais/definitivas) da rodada atual
# ================================

def fetch_pontuados(timeout=15):
    url = "https://api.cartola.globo.com/atletas/pontuados"
    try:
        r = requests.get(url, headers=HEADERS, timeout=timeout)
        if r.status_code != 200:
            return {}
        data = r.json()
        atletas = data.get("atletas", {}) if isinstance(data, dict) else {}
        if not isinstance(atletas, dict):
            return {}
        out = {}
        for k, v in atletas.items():
            try:
                atleta_id = int(k)
                pont = v.get("pontuacao") if isinstance(v, dict) else None
                if pont is None:
                    continue
                out[atleta_id] = float(pont)
            except Exception:
                continue
        return out
    except Exception:
        return {}


def fetch_time_payload(time_id: int, rodada: int, timeout=15):
    endpoints = [
        f"https://api.cartolafc.globo.com/time/id/{time_id}/{rodada}",
        f"https://api.cartolafc.globo.com/time/{time_id}/{rodada}",
    ]

    for url in endpoints:
        for tentativa in range(3):
            try:
                r = requests.get(url, headers=HEADERS, timeout=timeout)
                if r.status_code == 429:
                    time.sleep(0.8 + tentativa * 0.8)
                    continue
                if r.status_code != 200:
                    break
                data = r.json()
                if not isinstance(data, dict):
                    data = {}
                return data
            except Exception:
                time.sleep(0.5)
                continue
    return {}


def fetch_partidas_rodada(rodada: int, timeout=15):
    url = f"https://api.cartolafc.globo.com/partidas/{rodada}"
    try:
        r = requests.get(url, headers=HEADERS, timeout=timeout)
        if r.status_code != 200:
            return []
        data = r.json()
        if isinstance(data, dict):
            partidas = data.get("partidas", [])
            return partidas if isinstance(partidas, list) else []
    except Exception:
        pass
    return []


def clubes_que_ja_jogaram(rodada: int):
    partidas = fetch_partidas_rodada(rodada)
    clubes = set()
    for p in partidas:
        if not isinstance(p, dict):
            continue
        status = (p.get("status_transmissao_tr") or p.get("status_transmissao") or "").upper()
        encerrada = status in ("ENCERRADA", "FINALIZADA")
        if p.get("placar_oficial_mandante") is not None or p.get("placar_oficial_visitante") is not None:
            encerrada = True
        if encerrada:
            mid = p.get("clube_casa_id")
            vid = p.get("clube_visitante_id")
            try:
                if mid is not None:
                    clubes.add(int(mid))
            except Exception:
                pass
            try:
                if vid is not None:
                    clubes.add(int(vid))
            except Exception:
                pass
    return clubes


def setor_por_posicao(posicao_id: int) -> str:
    mapa = {
        1: "Goleiro",
        2: "Laterais",
        3: "Zagueiros",
        4: "Meias",
        5: "Atacantes",
        6: "Tecnico",
    }
    return mapa.get(int(posicao_id), "")


def _id_int(val):
    try:
        return int(val)
    except Exception:
        return None


def calcular_parcial_time(time_id: int, rodada: int, mapa_pontuados: dict, clubes_jogaram: set):
    data = fetch_time_payload(time_id, rodada)
    atletas = data.get("atletas") if isinstance(data, dict) else None
    if not isinstance(atletas, list):
        return 0.0

    titulares = []
    tecnico = None
    for a in atletas:
        if not isinstance(a, dict):
            continue
        pos = a.get("posicao_id")
        if pos == 6:
            tecnico = a
        else:
            titulares.append(a)

    luxo_id = data.get("reserva_luxo_id") if isinstance(data, dict) else None
    if luxo_id is not None:
        luxo_id = _id_int(luxo_id)

    # Capitao (1.5x)
    capitao_id = None
    try:
        capitao_id = data.get("capitao_id") if isinstance(data, dict) else None
        if capitao_id is None and isinstance(data.get("time"), dict):
            capitao_id = data["time"].get("capitao_id")
        capitao_id = _id_int(capitao_id)
    except Exception:
        capitao_id = None

    titulares_por_pos = {}
    for a in titulares:
        pos = a.get("posicao_id")
        titulares_por_pos.setdefault(pos, []).append(a)

    atletas_em_jogo = list(titulares)

    reservas = data.get("reservas", []) if isinstance(data, dict) else []
    if isinstance(reservas, list):
        for r in reservas:
            if not isinstance(r, dict):
                continue
            # reserva de luxo nao entra como banco
            if luxo_id is not None and _id_int(r.get("atleta_id")) == luxo_id:
                continue
            # banco so entra se reserva tiver pontuacao parcial positiva
            rid = _id_int(r.get("atleta_id"))
            if rid is None or rid not in mapa_pontuados:
                continue
            if float(mapa_pontuados.get(rid, 0.0)) <= 0.0:
                continue
            pos = r.get("posicao_id")
            if pos is None:
                continue
            candidatos = titulares_por_pos.get(pos, [])
            titular_sub = None
            for t in candidatos:
                tid = _id_int(t.get("atleta_id"))
                if tid is None:
                    continue
                clube_tid = _id_int(t.get("clube_id"))
                if tid not in mapa_pontuados and (clube_tid in clubes_jogaram):
                    titular_sub = t
                    break
            if titular_sub is None:
                continue
            try:
                atletas_em_jogo.remove(titular_sub)
            except ValueError:
                pass
            atletas_em_jogo.append(r)

    # Reserva de luxo
    luxo_capitao = False
    if luxo_id is not None:
        ids_em_jogo = set()
        for a in atletas_em_jogo:
            ids_em_jogo.add(_id_int(a.get("atleta_id")))
        if luxo_id not in ids_em_jogo:
            luxo_obj = None
            if isinstance(reservas, list):
                for r in reservas:
                    if isinstance(r, dict) and _id_int(r.get("atleta_id")) == luxo_id:
                        luxo_obj = r
                        break
            if isinstance(luxo_obj, dict):
                p_luxo = float(mapa_pontuados.get(luxo_id, 0.0))
                if p_luxo > 0:
                    setor_luxo = setor_por_posicao(luxo_obj.get("posicao_id"))
                    candidatos_setor = []
                    for a in atletas_em_jogo:
                        if setor_por_posicao(a.get("posicao_id")) == setor_luxo:
                            candidatos_setor.append(a)
                    if candidatos_setor:
                        # luxo so entra se TODOS do setor ja jogaram
                        todos_setor_jogaram = True
                        for c in candidatos_setor:
                            clube_c = _id_int(c.get("clube_id"))
                            if clube_c not in clubes_jogaram:
                                todos_setor_jogaram = False
                                break
                        if todos_setor_jogaram:
                            def pts(a):
                                try:
                                    return float(mapa_pontuados.get(int(a.get("atleta_id")), 0.0))
                                except Exception:
                                    return 0.0
                            pior_pts = None
                            piores = []
                            for c in candidatos_setor:
                                v = pts(c)
                                if pior_pts is None or v < pior_pts:
                                    pior_pts = v
                                    piores = [c]
                                elif v == pior_pts:
                                    piores.append(c)
                            pior = None
                            if len(piores) > 1 and capitao_id is not None:
                                for c in piores:
                                    if _id_int(c.get("atleta_id")) == capitao_id:
                                        pior = c
                                        break
                            if pior is None:
                                pior = piores[0] if piores else None
                            if pior is not None and p_luxo > pts(pior):
                                if capitao_id is not None and _id_int(pior.get("atleta_id")) == capitao_id:
                                    luxo_capitao = True
                                try:
                                    atletas_em_jogo.remove(pior)
                                except ValueError:
                                    pass
                                atletas_em_jogo.append(luxo_obj)

    total = 0.0
    for a in atletas_em_jogo:
        try:
            aid = int(a.get("atleta_id"))
        except Exception:
            continue
        total += float(mapa_pontuados.get(aid, 0.0))

    # Bonus do capitao (50%)
    if capitao_id is not None:
        cap_pts = float(mapa_pontuados.get(capitao_id, 0.0))
        if cap_pts != 0.0:
            ids_em_jogo = set()
            for a in atletas_em_jogo:
                ids_em_jogo.add(_id_int(a.get("atleta_id")))
            if capitao_id in ids_em_jogo:
                total += cap_pts * 0.5
            elif luxo_capitao and luxo_id is not None and luxo_id in ids_em_jogo:
                luxo_pts = float(mapa_pontuados.get(luxo_id, 0.0))
                if luxo_pts != 0.0:
                    total += luxo_pts * 0.5

    if isinstance(tecnico, dict):
        try:
            tid = int(tecnico.get("atleta_id"))
            total += float(mapa_pontuados.get(tid, 0.0))
        except Exception:
            pass

    return round(total, 2)


def status_mercado_e_rodada():
    try:
        m = api.mercado()
        return int(getattr(m, "status_mercado", 0)), int(getattr(m, "rodada_atual", 0))
    except Exception:
        try:
            r = requests.get("https://api.cartola.globo.com/mercado/status", headers=HEADERS, timeout=15)
            if r.status_code != 200:
                return 0, 0
            data = r.json()
            return int(data.get("status_mercado", 0)), int(data.get("rodada_atual", 0))
        except Exception:
            return 0, 0


status_http, rodada_ref = status_mercado_e_rodada()
FASES_POR_RODADA = {
    1: "primeira_fase",
    2: "oitavas",
    3: "quartas",
    4: "semi",
}

fase_atual = FASES_POR_RODADA.get(rodada_ref)

pontuacoes_rodada = {}
ids_validos = [_id_int(x) for x in ids_participantes if _id_int(x) is not None]
ids_validos = list(dict.fromkeys(ids_validos))
mapa_pontuados = {}

if rodada_ref and INICIO_COPA <= rodada_ref <= FIM_COPA and ids_validos:
    # usa parciais se houver pontuados, independentemente do status_http
    mapa_pontuados = fetch_pontuados()
    if mapa_pontuados:
        clubes_jogaram = clubes_que_ja_jogaram(rodada_ref)
        for tid in ids_validos:
            try:
                pontuacoes_rodada[tid] = calcular_parcial_time(tid, rodada_ref, mapa_pontuados, clubes_jogaram)
            except Exception:
                continue
    else:
        if status_http == 2:
            print("Parciais vazias no momento.")
        for tid in ids_validos:
            try:
                t = api.time(time_id=tid, rodada=rodada_ref)
                pontuacoes_rodada[tid] = float(getattr(t, "ultima_pontuacao", 0.0))
            except Exception:
                continue

print(f"Status={status_http} | rodada_ref={rodada_ref} | fase_atual={fase_atual} | parciais={'sim' if mapa_pontuados else 'nao'}")



Status=0 | rodada_ref=1 | fase_atual=primeira_fase | parciais=sim


In [7]:
# GERAR copa_dados.js
# Ajuste a ordem dos confrontos aqui quando o Cartola divulgar.
import json
from pathlib import Path

# Mapa id -> nome (a partir do df_base)
mapa_nomes = df_base.reset_index().set_index('time_id')['Time'].to_dict() if not df_base.empty else {}

# Normaliza lista de IDs (32 times)
lista_ids = []
for i, raw in enumerate(ids_participantes):
    try:
        v = int(raw)
        lista_ids.append(v)
    except Exception:
        lista_ids.append(None)

if len(lista_ids) < 32:
    lista_ids.extend([None] * (32 - len(lista_ids)))

# Times (ordem conforme ids_participantes)
times = []
for i, time_id in enumerate(lista_ids[:32]):
    if isinstance(time_id, int):
        nome = mapa_nomes.get(time_id, f"Time {i+1}")
    else:
        nome = f"Time {i+1}"
    times.append({"id": time_id, "nome": nome, "turnoPts": None})

# Confrontos da Primeira Fase (preencher manualmente)
# Formato: (casaId, foraId)
confrontos_primeira_fase = [
    (25811332, 14124559),
    (28741323, 51010813),
    (29228373, 49180400),
    (3447341, 212042),
    (13913874, 3424598),
    (117598, 18344271),
    (48498051, 3708025),
    (25565675, 18223508),
    (24468241, 13951133),
    (24856400, 1747619),
    (20696550, 528730),
    (335716, 5823700),
    (18661583, 186283),
    (13707047, 18642587),
    (3851966, 18346776),
    (479510, 19033717),
]


# Monta confrontos com pontuacoes se a fase estiver ativa
def _montar_confrontos(lista, fase_key):
    out = []
    for casa, fora in lista:
        if fase_key == fase_atual and pontuacoes_rodada:
            casa_pts = pontuacoes_rodada.get(casa)
            fora_pts = pontuacoes_rodada.get(fora)
        else:
            casa_pts = None
            fora_pts = None
        out.append({"casaId": casa, "foraId": fora, "casaPts": casa_pts, "foraPts": fora_pts})
    return out

copa_dados = {
    "temporada": "2026/1",
    "times": times,
    "fases": {
        "primeira_fase": _montar_confrontos(confrontos_primeira_fase, "primeira_fase"),
        "oitavas": [],
        "quartas": [],
        "semi": [],
        "final": [],
        "terceiro": [],
    },
}

saida = Path('copa_dados_leon.js')
conteudo_js = "window.copaDados = " + json.dumps(copa_dados, ensure_ascii=False, indent=2) + ";"

try:
    rodada_meta = int(rodada_ref)
except Exception:
    rodada_meta = None

try:
    parcial_disponivel = bool(mapa_pontuados)
except Exception:
    parcial_disponivel = False

meta = {"rodada": rodada_meta, "parcial_disponivel": parcial_disponivel}
conteudo_js = conteudo_js + "" + "window.copaMeta = " + json.dumps(meta, ensure_ascii=False, indent=2) + ";"

saida.write_text(conteudo_js, encoding='utf-8')
print(f'Arquivo gerado: {saida.resolve()}')





Arquivo gerado: C:\Users\ferna\Projetos\GitHub\cartola_2026\copa_leon\datasets_copa_leon\copa_dados_leon.js
